# (Meta-)Dynamics of rotor systems

run with dftb_plumed conda environment

In [ ]:
from ase.visualize import view
from ase.io import read

atoms = read('../data/structures/new_systems/ktu_002.cif')

# view(atoms)
# len(atoms)

In [ ]:
# !ulimit -s unlimited
# conda env export > dftb_plumed.yml
# https://wiki.fysik.dtu.dk/ase/ase/calculators/dftb.html
# https://dftb.org/parameters/download

# export ASE_DFTB_COMMAND="/path/to/dftb+ > PREFIX.out"
# export DFTB_PREFIX=/path/to/mio-0-1/
import os

os.environ['OMP_NUM_THREADS'] = "6,1"
os.environ["ASE_DFTB_COMMAND"] = "ulimit -s unlimited; /home/dlb/anaconda3/envs/dftb_plumed/bin/dftb+ > PREFIX.out"
os.environ["DFTB_PREFIX"] = "/home/dlb/Downloads/pbc-0-3"
# os.environ["DFTB_PREFIX"] = "/home/dlb/Downloads/auorgap-1-1" # doesn't work for now with crystal...

# import os

# os.environ["ASE_DFTB_COMMAND"] = "/home/dlb/anaconda3/envs/qe/bin/dftb+"
# os.environ["DFTB_PREFIX"] = "/home/dlb/Downloads/pbc-0-3"

In [ ]:
# %%capture cap

%load_ext autoreload
%autoreload 2


# https://www.plumed.org/doc-v2.7/user-doc/html/_m_e_t_a_d.html
# https://github.com/dftbplus/dftbplus/issues/1064
# try with numpy 1.22　
# https://gitlab.com/Sucerquia/ase-plumed_tutorial

from ase.calculators.lj import LennardJones
# from ase.calculators.plumed import Plumed
from runner.plumed import Plumed
# from plumed import Plumed
from ase.constraints import FixedPlane
from ase.io import read
from ase import units

from ase.geometry.analysis import Analysis
from ase.constraints import FixAtoms, FixBondLengths

# import plumed

from ase.calculators.dftb import Dftb



ps = 1000 * units.fs
timestep = 0.1 # in what units? 

# height 30.50 is fast

setup = [f"UNITS LENGTH=A TIME={1/ps} ENERGY={units.mol/units.kJ}",
        #  "d: DISTANCE ATOMS=1,2 ",
        "at8: FIXEDATOM AT=-0.3652684296711638,5.59639757164289,11.089411052917308",
        "at6: FIXEDATOM AT=-0.5388786945959876,6.471914514609469,10.07028636148657",
        "c1: CENTER ATOMS=at8,at6",
        "c2: CENTER ATOMS=67,69,73,77,79,83",
        "c3: CENTER ATOMS=7,9,11,13,15,17",
        "c4: CENTER ATOMS=79,83",
         "phi: TORSION ATOMS=c1,c2,c3,c4",
         "metad: METAD ARG=phi SIGMA=1.10 HEIGHT=15.0 BIASFACTOR=5 TEMP=300.0 PACE=1", #GRID_MIN=-pi GRID_MAX=pi GRID_BIN=1500
         # "ARG=d SIGMA=0.20,0.20 HEIGHT=1.20 BIASFACTOR=5 TEMP=300.0 PACE=500",
         # "GRID_MIN=0,0 GRID_MAX=1.0,1.0 GRID_BIN=150,150",
         "PRINT ARG=phi STRIDE=1 FILE=../results/test/plumed/COLVAR",
         # "CALC_RCT ",
         # "RCT_USTRIDE=10",
         # "...",
         "FLUSH STRIDE=1000"]

# setup = f"""
# d: DISTANCE ATOMS=1,2 
# metad: METAD ...
#    ARG=d SIGMA=0.20,0.20 HEIGHT=1.20 BIASFACTOR=5 TEMP=300.0 PACE=500 
#    GRID_MIN=0,0 GRID_MAX=1.0,1.0 GRID_BIN=150,150 
#    CALC_RCT 
#    RCT_USTRIDE=10 
# ...
# PRINT ARG=d STRIDE=100 FILE=COLVAR
# """
# print(setup)

from ase.build import molecule

atoms = read('../data/structures/new_systems/ktu_002.cif')
# atoms = molecule('CH3CH2NH2')
# atoms = read('../structures/Ar_isomer.xyz')
# atoms = read('../structures/KTU-183_2_auto.cif')
# atoms=read("../structures/HIK-143 293K-activated.cif")
# atoms=read("../structures/KTU-183_2_auto_dimer.xyz")
# cons = [FixedPlane(i, [0, 0, 1]) for i in range(7)]
# atoms.set_constraint(cons)
# atoms.set_masses([1, 1, 1, 1, 1, 1, 1])

# calc_base = LennardJones(rc=2.5, r0=3.)


fix_atoms = FixAtoms(indices=[atom.index for atom in atoms if atom.symbol=='N' or atom.symbol=='Si' or atom.symbol=='O'])
# # ch_bonds = Analysis(atoms).get_bonds("C", "H")[0]
# # fix_bond_lengths = FixBondLengths(ch_bonds)
# # atoms.set_constraint([fix_atoms, fix_bond_lengths])
atoms.set_constraint(fix_atoms)



calc_base = Dftb(atoms=atoms,
            label='crystal',
            # Hamiltonian_ = "xTB",
            # Hamiltonian_Method = "GFN1-xTB",
            # Hamiltonian_MaxAngularMomentum_='',
            # Hamiltonian_MaxAngularMomentum_O='p',
            # Hamiltonian_MaxAngularMomentum_H='s',
            # Hamiltonian_MaxAngularMomentum_N='s',
            # Hamiltonian_MaxAngularMomentum_C='s',
            # Hamiltonian_MaxAngularMomentum_Si='s',
            kpts=(1,1,1),
            # Hamiltonian_SCC='Yes',
            # Verbosity=0,
            # Hamiltonian_OrbitalResolvedSCC = 'Yes',
            # Hamiltonian_SCCTolerance=1e-15,
            # kpts=None
            # Driver_='ConjugateGradient',
            # Driver_MaxForceComponent=1e-3,
            # Driver_MaxSteps=200,
            # Driver_LatticeOpt = 'Yes',
            # Driver_AppendGeometries = 'Yes'
            )

atoms.calc = Plumed(calc=calc_base,
                    input=setup,
                    timestep=timestep,
                    atoms=atoms,
                    kT=0.1,
                    log='../results/test/plumed/plumed_log.log')

from ase.md.verlet import VelocityVerlet
from ase.md.langevin import Langevin

# atoms.calc = calc_base

# calc=LennardJones(rc=2.5, r0=3.)
# atoms.calc = calc



dyn = Langevin(atoms, timestep, temperature_K=0.1/units.kB, friction=0.1,
               fixcm=True, trajectory='../results/test/plumed/MTD_output.xyz')
# We want to run MD with constant energy using the VelocityVerlet algorithm.
# dyn = VelocityVerlet(atoms, timestep, trajectory='../results/test/plumed/UnbiasMD.xyz')  # 5 fs time step.
# https://wiki.fysik.dtu.dk/ase/ase/md.html

dyn.run(500)


In [ ]:
from ase.visualize import view
view(atoms)



In [ ]:
print(atoms[6])
print(atoms[8])
print(atoms.get_cell())

In [ ]:
from ase.io import read, write
from ase.visualize import view

traj = read('../results/test/plumed/MTD_output.xyz', index=":")
# traj = read('/home/dlb/Downloads/UnbiasMD.xyz', index=":")
# write("test/plumed/biased_MTD.xyz", traj, format="extxyz")
# traj_biased = read('test/plumed/biased_MTD.xyz', index=":")
# traj_unbiased = read('test/plumed/.xyz', index=":")

view(traj)

In [ ]:
traj[1].get_forces()

In [ ]:
# del test_at

test_at = traj[1].copy()


test_at.calc = calc_base

# print(test_at.get_potential_energy())
print(test_at.get_forces())

In [ ]:
# # Get unbiased forces:

# traj_unbiased = read('test/plumed/biased_MTD.xyz', index=":")

# for im in traj_unbiased:
#         im.calc = calc_base
#         # im.get_forces()
#         print(im.get_forces())

# write("test/plumed/unbiased_MTD.xyz", traj_unbiased, format="extxyz")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

traj_biased = read('test/plumed/biased_MTD.xyz', index=":")
traj_unbiased = read('test/plumed/unbiased_MTD.xyz', index=":")

f_diff = np.zeros((len(traj_biased), len(traj_biased[0]), 3))
e_diff = np.zeros((len(traj_biased)))

for i in range(len(traj_biased)):
        f_diff[i] = traj_biased[i].get_forces() - traj_unbiased[i].get_forces()
        e_diff[i] = traj_biased[i].get_potential_energy() - traj_unbiased[i].get_potential_energy()


        # print(f_diff)

force_diffs = np.absolute(f_diff).sum(axis=(1,2))
plt.plot(force_diffs)
plt.show()


energy_diffs = np.absolute(e_diff)
plt.plot(energy_diffs)
plt.show()
# f_diff[0].absolutize().sum()

## Extract connected components

In [ ]:
from ase.build import connected_indices, connected_atoms, separate, split_bond
# view(connected_atoms(atoms, 0))
# separate(atoms)
view(split_bond(atoms, 0, 1))

## Make supercell

In [ ]:
from ase.build import make_supercell

new_cryst = make_supercell(crystal,[[2,0,0], [0,2,0], [0,0,2]])
print(new_cryst)
view(new_cryst)